In [ ]:
%pprint

import sys
import os
import re

# Check if running in container (check for /workspace or /.dockerenv)
if os.path.exists('/workspace') or os.path.exists('/.dockerenv'):
    sys.path.insert(1, '/workspace')
    # Ensure we're in the correct year directory for relative input paths
    if os.path.basename(os.getcwd()) != '2025':
        os.chdir('/workspace/2025')
else:
    sys.path.insert(1, '..')
    # When outside container, change to the notebook's directory (2025/)
    notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
    # If we're not in 2025 directory, navigate there
    if os.path.basename(os.getcwd()) != '2025':
        year_dir = os.path.join(os.path.dirname(os.path.abspath(sys.path[1])), '2025')
        if os.path.exists(year_dir):
            os.chdir(year_dir)
    
from aoc_utils import *
from bigtree import Node, list_to_tree, levelorder_iter



### “i.e.” Latin "id est" => “that is.”  
### “e.g.” Latin "exempli gratia" => “for example.”

# Home

Each day's work will consist of three tasks:
- **Input**: Parse the day's input file with the function `parse(day, parser, sep)`, which treats the input as a sequence of *entries*, separated by `sep` (default newline); applies `parser` to each entry; and returns the results as a tuple. (Note: `ints` and `atoms` are useful `parser` functions (as are `int` and `str`).)
- **Part 1**: Write code to compute the answer to Part 1, and submit the answer to the AoC site. Use the function `answer` to record the correct answer and serve as a regression test when I re-run the notebook.
- **Part 2**: Repeat coding and `answer` for Part 2.

1. [Day 1](#day-1)
2. [Day 2](#day-2)
3. [Day 3](#day-3)
4. [Day 4](#day-4)
5. [Day 5](#day-5)
6. [Day 6](#day-6)
7. [Day 7](#day-7)
8. [Day 8](#day-8)
9. [Day 9](#day-9)
10. [Day 10](#day-10)
11. [Day 11](#day-11)
12. [Day 12](#day-12)


[home](#home)
# Day 1
[Secret Entrance](https://adventofcode.com/2025/day/1)  
```
```

In [ ]:
get_in_file(1,2025)

In [ ]:
in_part_A = Input(1)
in_part_A

In [ ]:
regex = r"""(?P<dir>[LR])(?P<click>-?\d+)"""
with open("input/input1.txt") as f:
    match = re.finditer(regex, f.read())
    l = [(m.group('dir'),int(m.group('click'))) for m in match]
l
# one line version
ll = [(dir,int(click)) for (dir,click) in collapse([re.findall(regex, line) for line in in_part_A], base_type=tuple)]
ll == l

In [ ]:
# L sub R adds
def compute_positions(l):
    start = 50
    for dir, click in l:
        if dir == 'L':
            # turn left
            start = (start - click) % 100
        else:
            # turn right
            start = (start + click) % 100
        yield start

In [ ]:
res_a = Counter(list(compute_positions(l)))[0]
res_a

In [ ]:
submit(res_a, part="a", day=1, year=2025)

### Part 2

In [ ]:
test = """
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
"""

match = re.finditer(regex, test)
t = [(m.group('dir'),int(m.group('click'))) for m in match]
t

In [ ]:
# L sub R adds
from pydoc import cli
from turtle import st


def compute_positions_and_cross(l):
    start = 50
    for dir, click in l:
        cross = 0
        loops = click // 100
        click = click % 100
        if dir == 'L':
            # turn left
            cross = 1 if ((start - click) < 0 and start != 0 ) else 0  
            start = (start - click) % 100
        else:
            # turn right
            cross = 1 if ((start + click) > 100 and start != 0 ) else 0
            start = (start + click) % 100
            
        yield start,cross+loops
    

In [ ]:
dial,cross = unzip(compute_positions_and_cross(l))

In [ ]:
res_b = Counter(dial)[0] + sum(cross)
res_b

In [ ]:
submit(res_b, part="b", day=1, year=2025)

[home](#home)
# Day 2
[Gift Shop](https://adventofcode.com/2025/day/2)  
```
```

In [ ]:
get_in_file(2,2025)

In [ ]:
test = """
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124
"""

In [ ]:
parse(2,sep=",")

In [ ]:
# get IDs ranges as tuples from input file
regex = r"""(?P<first_ID>\d+)-(?P<last_ID>\d+)"""
with open("input/input2.txt") as f:
    match = re.finditer(regex, f.read())
    ids_range = tuple((int(m.group('first_ID')),int(m.group('last_ID'))) for m in match)
ids_range

In [ ]:
# one line version
ll = tuple((int(_),int(__)) for (_,__) in collapse([re.findall(regex, line) for line in parse(2,sep=",")], base_type=tuple))
ll == ids_range

In [ ]:
ttest = tuple((int(_),int(__)) for (_,__) in collapse([re.findall(regex, line) for line in test.split(",")], base_type=tuple))
ttest

In [ ]:
def is_invalid_ID(ID):
    sID = str(ID)
    # Check for even string length
    if len(sID) % 2 != 0:
        return False
    # split in half and compare
    half = len(sID) // 2
    if sID[:half] != sID[half:]:
        return False
    return True
    
def gen_invalid_IDs(first_ID, last_ID, filter_IDs_func=is_invalid_ID):
    for _id in range(first_ID, last_ID+1):
        if filter_IDs_func(_id):
            yield _id

invalid_IDs = list(collapse([gen_invalid_IDs(*ids) for ids in ids_range]))
invalid_IDs


In [ ]:
res_a = sum(invalid_IDs)
res_a

In [ ]:
submit(res_a, part="a", day=2, year=2025)

### Part 2

In [ ]:
# finds any repeated pattern
pattern = r"""(?P<pattern>(\d+))\1+"""

In [ ]:
re.fullmatch(pattern, "113447113447").groups()

In [ ]:
def is_still_invalid_ID(ID):
    sID = str(ID)
    if re.fullmatch(pattern, sID):
        return True
    return False

invalid_IDs = list(collapse([gen_invalid_IDs(*ids,filter_IDs_func=is_still_invalid_ID) for ids in ids_range]))
invalid_IDs


In [ ]:
res_b = sum(invalid_IDs)
res_b

In [ ]:
submit(res_b, part="b", day=2, year=2025)

[home](#home)
# Day 3
[Lobby](https://adventofcode.com/2025/day/3)  
```
```

In [ ]:
get_in_file(3,2025)

In [ ]:
batt_banks = Input(3)
batt_banks

In [ ]:
def max_bank_joltage (bank) :
    """
    exactly two batteries can be used from the bank to get the maximum joltage
    """
    i = argmax(bank)
    if i == len(bank)-1 :
        i = argmax(bank[:-1])
    if i == 0 :
        j = argmax(bank[1:])
        j += 1
    else :
        j = argmax(bank[i+1:])
        j += i + 1
    return int(bank[i]+bank[j])

In [ ]:
res_a = sum([max_bank_joltage(bank) for bank in batt_banks])
res_a

In [ ]:
submit(res_a, part="a", day=3, year=2025)

### Part 2

Use a greedy “remove k digits” subsequence algorithm.

Think “remove exactly k digits” rather than “pick 12 peaks.” You want the lexicographically largest subsequence of length 12.

Use a stack:
- Iterate left to right; while you can still remove digits and the top of the stack is less than the current digit, pop it.
- Push the current digit.
- If you still have removals left at the end, drop them from the tail.
- Slice the result to the first 12 digits to handle any surplus.


In [ ]:
test = """
987654321111111
811111111111119
234234234234278
818181911112111
"""

In [ ]:
test_banks = [mapt(int, list(line.strip())) for line in test.strip().split("\n")]
test_banks

In [ ]:
def max_bank_joltage_12batt(bank):
    """
    Select exactly 12 batteries (in order) to maximize the 12-digit joltage.
    Greedy approach: remove k = len(bank) - 12 digits to form the lexicographically
    largest subsequence of length 12.
    """
    target = 12
    n = len(bank)
    if n <= target:
        return int("".join(map(str, bank)))

    k = n - target  # digits to remove
    stack = []
    for d in bank:
        while k > 0 and stack and stack[-1] < d:
            stack.pop()
            k -= 1
        stack.append(d)

    # If removals remain, drop from the end
    if k > 0:
        stack = stack[:-k]

    selected = stack[:target]
    return int("".join(map(str, selected)))


In [ ]:
[max_bank_joltage_12batt(bank) for bank in test_banks]

In [ ]:
res_b = sum([max_bank_joltage_12batt(bank) for bank in test_banks])
res_b

In [ ]:
res_b = sum([max_bank_joltage_12batt(bank) for bank in batt_banks])
res_b

In [ ]:
submit(res_b, part="b", day=3, year=2025)

[home](#home)
# Day 4
[Printing Department](https://adventofcode.com/2025/day/4)  
```
```

In [ ]:
get_in_file(4,2025)

In [ ]:
test_d4 ="""
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
"""

In [ ]:
in_part_A = mapt(lambda x: tuple(x), Input(4))
#in_part_A = mapt(lambda x: tuple(x), test_d4.rstrip().split())
M = np.matrix(in_part_A)
shape = M.shape
shape

In [ ]:
len(np.argwhere(M == '@'))

In [ ]:
def on_border(i,j,shape):
    if i == 0 or j == 0 or i == shape[0]-1 or j == shape[1]-1 :
        return True
    return False

res_a = 0
with np.nditer(M, flags=['multi_index']) as it :
    for x in it:
        #print("%s <%s>" % (x, it.multi_index))
        #if on_border(*it.multi_index, M.shape) :
        #    continue
        if x != '@' :
            # next matrix element
            continue
        # found '@' ... check neighbors
        c = Counter()
        for nk, nv in get_adj(*it.multi_index, M).items():
            c[str(nv)] +=1
        if c['@'] < 4 :
            res_a +=1  
res_a   

In [ ]:
submit(res_a, part="a", day=4, year=2025)

### Part 2

In [ ]:
def forklift_cycle(_M):
    to_remove = []
    with np.nditer(_M, flags=['multi_index']) as it :
        for x in it:
            if x != '@' :
                # next matrix element
                continue
            # found '@' ... check neighbors
            c = Counter()
            for nk, nv in get_adj(*it.multi_index, _M).items():
                c[str(nv)] +=1
            if c['@'] < 4 :
                to_remove.append(it.multi_index)
    return to_remove

len(forklift_cycle(M))

In [ ]:
def forklift_move_all(_M):
    M_copy = _M.copy()
    while True :
        to_remove = forklift_cycle(M_copy)
        if len(to_remove) == 0 :
            break
        yield len(to_remove)
        for i,j in to_remove :
            M_copy[i,j] = '.'

In [ ]:
res_b = sum(forklift_move_all(M))
res_b

In [ ]:
submit(res_b, part="b", day=4, year=2025)

[home](#home)
# Day 5
[Cafeteria](https://adventofcode.com/2025/day/5)  
```
```

In [ ]:
get_in_file(5,2025)

In [ ]:
test_d5 ="""
3-5
10-14
16-20
12-18

1
5
8
11
17
32
"""

In [ ]:
p = lambda x: x.strip().split()
id_ranges, iDs = parse(5, parser=p, sep="\n\n")
#id_ranges, iDs = mapt(p, test_d5.split("\n\n"))
iDs = tuple(sorted(mapt(int, iDs)))
id_ranges = tuple(sorted(mapt(lambda x: mapt(int,x),(mapt(lambda x: x.split('-'),id_ranges)))))
id_ranges


In [ ]:
def overlap(r1, r2):
    A, B = r1
    C, D = r2
    if A == C :
        return (A, max(B,D)),True
    if B < C :
        return r2,False
    elif B < D :
        return (A,D),True
    else :
        return (A,B),True  

def merge_ranges(ranges):
    merged = []
    current = ranges[0]
    for r in ranges[1:]:
        new_range, is_overlap = overlap(current, r)
        if not is_overlap :
            # no overlap
            merged.append(current)
            current = r
        else:
            current = new_range
    merged.append(current)
    return merged

merged_id_ranges = merge_ranges(id_ranges)
merged_id_ranges


In [ ]:
iDs

In [ ]:
def in_ranges(x, ranges):
    return any(a <= x <= b for a, b in ranges)

fresh = [id_ for id_ in iDs if in_ranges(id_, merged_id_ranges)]
fresh

In [ ]:
res_a = len(fresh)
res_a

In [ ]:
submit(res_a, part="a", day=5, year=2025)

### Part 2

In [ ]:
merged_id_ranges

In [ ]:
len(merged_id_ranges)

In [ ]:
res_b = sum(len(range(a, b+1)) for (a,b) in merged_id_ranges)
res_b

In [ ]:
submit(res_b, part="b", day=5, year=2025)

[home](#home)
# Day 6
[Trash Compactor](https://adventofcode.com/2025/day/6)  
```
```

In [ ]:
get_in_file(6,2025)

In [ ]:
test_d6 ="""
123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
"""

In [ ]:
*nums,op=Input(6)
#*nums,op = test_d6.strip().splitlines()

In [ ]:
M_num = np.matrix(mapt(lambda x: mapt(int, x.split()), nums))
V_op = np.array((op.split()))
M_num.T

In [ ]:
V_op

In [ ]:
res_a = sum([np.prod(n) if op == '*' else np.sum(n) for n, op in zip(M_num.T, V_op)])
res_a

In [ ]:
submit(res_a, part="a", day=6, year=2025)

### Part 2

In [ ]:
M_num = np.matrix([tuple(x.replace(' ', '.').strip()) for x in nums])
M_num

In [ ]:
def gen_matrix_slice_on_dots(_M):
    # col idx where are all '.'
    split_points = np.flatnonzero(np.all(_M == '.', axis=0))
    start = 0
    for idx in split_points:
        if idx > start:
            yield _M[:, start:idx]
        start = idx + 1
    # trailing block
    if start < _M.shape[1]:
        yield _M[:, start:]
        
# Extract numbers from each row (remove dots), apply operation
results = []
for op, block in zip(V_op, gen_matrix_slice_on_dots(M_num)):
    print(f"Block:\n{block.T}")
    # Convert each row to a 1D list to avoid numpy ambiguity
    numbers = [int(''.join([x for x in np.asarray(row).flatten().tolist() if x != '.'])) for row in block.T]
    if op == '+':
        result = sum(numbers)
    elif op == '*':
        result = math.prod(numbers)
    else:
        raise ValueError(f"Unknown operation: {op}")
    results.append(result)
    print(f"Op: {op}, Numbers: {numbers}, Result: {result}")

res_b = sum(results)
res_b 

In [ ]:
submit(res_b, part="b", day=6, year=2025)

[home](#home)
# Day 7
[Laboratories](https://adventofcode.com/2025/day/7)  
```
```

In [ ]:
get_in_file(7,2025)

In [ ]:
test_d7 = """
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
"""

In [ ]:
in_part_A = mapt(lambda x: tuple(x), Input(7))
#in_part_A = mapt(lambda x: tuple(x), test_d7.rstrip().split())
M = np.matrix(in_part_A)
shape = M.shape
shape

In [ ]:
start= np.argwhere(M == 'S').tolist()[0]
start

In [ ]:
splitters = {tuple(s) for s in np.argwhere(M == '^').tolist()}
splitters

In [ ]:
def beam_splitting(_M, start, splitters):
    visited = set()
    down = (1,0)
    left = (0,-1)
    right = (0,1)
    to_visit = [add_tuple(start, down)]
    
    while to_visit:
        current = to_visit.pop()
        if current in visited:
            continue
        visited.add(current)
        if current in splitters:
            # Split beam left and right
            if isValid(_M.shape, add_tuple(current, left)) :
                to_visit.append(add_tuple(current, left))
            if isValid(_M.shape, add_tuple(current, right)) :
                to_visit.append(add_tuple(current, right))
            continue
        elif _M[current] == '.':
            _M[current] = '|'
            if isValid(_M.shape, add_tuple(current, down)) :
                to_visit.append(add_tuple(current, down))
            continue
        else :
            # Continue downward
            if isValid(_M.shape, add_tuple(current, down)) :
                to_visit.append(add_tuple(current, down))
            
    return _M
    
M_result = beam_splitting(M.copy(), tuple(start), splitters)
M_result

In [ ]:
end_test_d7 = """
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
"""
M = np.matrix(mapt(lambda x: tuple(x), end_test_d7.rstrip().split()))
len(np.argwhere(M == '|').tolist())

In [ ]:
def count_splits(_M):
    # Alternative: use string matching per row (cleaner)
    _M_T = _M.T
    pattern_str = '|^'
    count = 0

    for row in _M_T:
        row_str = ''.join(row.A1)  # .A1 flattens matrix row to 1D array
        count += row_str.count(pattern_str)

    return count

In [ ]:
splits =count_splits(M_result)
splits

In [ ]:
res_a = splits
submit(res_a, part="a", day=7, year=2025)

### Part 2